## [실습2]

- 데이터베이스 시스템: 운영체제와 동급의 개념
    - 데이터베이스의 테이블은 윈도우가 다루는 파일(탐색기로 관리하는)의 개념과 동급임
    - 그래서 데이터베이스 테이블을 외부 파일로 내보낸다는 설명이 일반적임

    - DB를 만들고 테이블을 만들고 테이블의 정보를 crud 작업하는 것이, 운영체제 측면에서 디스크를 파티션 나누고, 폴더를 만들고 그 안에 파일을 대상으로 crud하는 것과 비슷한 흐름임

    - 트리거, 뷰

## 10.5 File to DB

### 10.5.1 csv to table
- csv파일: 텍스트파일인 셈

In [2]:
'''
csv >> db table
    1차실행: table생성 > 레코드 추가
    2차 실행: 레코드 검색
'''

# (1) csv파일 로드

import pandas as pd
import pymysql

bmi = pd.read_csv('Data/230614/bmi.csv')

# (2) 각 칼럼 추출
height = bmi['height']
weight = bmi['weight']
label = bmi['label']

print(bmi.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   height  20000 non-null  int64 
 1   weight  20000 non-null  int64 
 2   label   20000 non-null  object
dtypes: int64(2), object(1)
memory usage: 468.9+ KB
None


In [3]:
bmi.head(3)

,height,weight,label
0,184,61,thin
1,189,56,thin
2,183,79,normal


아래내용
- 1차 실행 시 테이블을 만들고 csv파일 자료에서 100개의 행을 테이블 레코드에 저장
- 2차 실행 시 저장된 레코드 100개가 검색됨

In [3]:
# (3) DB테이블로 만들기

config = {
    'host': '127.0.0.1',
    'user': 'root',
    'passwd': '0000',
    'database': 'test_db',
    'port': 3306,
    'charset': 'utf8',
    'use_unicode': True
}

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()

    # (3-1) table 조회
    cursor.execute('show tables')
    tables = cursor.fetchall()
    print(type(tables))

    # (3-2) 스위칭기법: 반복문 수행 과정에서 특정한 상태로 바뀔 때, 상태가 변경되었음을 외부에 알리는 기법
    sw = False
    for table in tables:
        if table[0] == 'bmi_tab':    ## 이중 tuple구문. table[0]이 bmi_tab인 경우,
            sw = True    # table이 있는 경우 swapping

    # (3-3) 테이블 없으면, 생성
    if not sw:
        print('테이블 없음')
        sql = '''
        create table bmi_tab(
        height int not null,
        weight int not null,
        label varchar(15) not null
        )
        '''
        cursor.execute(sql)
    
    # (3-4) 레코드 조회
    cursor.execute('select * from bmi_tab')
    rows = cursor.fetchall()

    if rows:
        for row in rows:
            print(f'{row[0]}    {row[1]}    {row[2]}')
        print('전체 레코드 수: ', len(rows))
    else:
        print('100 레코드 추가')
        for i in range(100):
            h = height[i]
            w = weight[i]
            lab = label[i]
            cursor.execute(f'insert into bmi_tab values({h}, {w}, "{lab}")')
            conn.commit()
except Exception as e:
    print("DB Error: ", e)
finally:
    cursor.close()
    conn.close()

<class 'tuple'>
184    61    thin
189    56    thin
183    79    normal
143    40    normal
187    66    normal
161    52    normal
186    54    thin
144    57    fat
184    55    thin
165    76    fat
143    45    normal
154    61    fat
159    57    normal
177    45    thin
174    78    fat
140    80    fat
150    75    fat
141    47    normal
186    57    thin
154    83    fat
141    65    fat
178    74    normal
157    84    fat
175    56    thin
172    40    thin
170    69    normal
143    64    fat
148    68    fat
161    56    normal
176    77    normal
142    60    fat
145    41    normal
179    84    fat
164    80    fat
153    53    normal
182    45    thin
167    49    thin
168    51    thin
155    40    thin
188    51    thin
159    44    thin
173    76    fat
168    64    normal
142    51    fat
179    68    normal
145    44    normal
140    69    fat
162    77    fat
185    59    thin
174    50    thin
147    69    fat
145    41    normal
149    68    fat
157    46    nor

In [4]:
print(tables)

(('bmi_tab',), ('goods',), ('labels',), ('tb1',))


In [5]:
type(tables)

tuple